In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from numpy import concatenate, mean, asarray
from svmutil import *
import time

# load the MNIST data by TensorFlow
mnist=input_data.read_data_sets("MNIST_data/", one_hot=False)

image_train=mnist.train.images
image_validation=mnist.validation.images
image_test=mnist.test.images

label_train=mnist.train.labels
label_validation=mnist.validation.labels
label_test=mnist.test.labels

# merge the training and validation datasets
image_train=concatenate((image_train, image_validation), axis=0)
label_train=concatenate((label_train, label_validation), axis=0)

# array to list
image_train=image_train.tolist()
image_test=image_test.tolist()
label_train=label_train.tolist()
label_test=label_test.tolist()

image_train=image_train[:1000]
image_test=image_test[:300]
label_train=label_train[:1000]
label_test=label_test[:300]

# record time
time_start=time.time() 

# linear SVM by Libsvm in Python
model=svm_train(label_train,image_train,'-t 0')
label_predict, accuracy, decision_values=svm_predict(label_test,image_test,model)

# # int to float
# label_predict=[int(tmp) for tmp in label_predict] 

# # list to array
# label_predict=asarray(label_predict)
# label_test=asarray(label_test)

# # accuracy
# accuracy=mean((label_predict==label_test)*1)
# print('Accuracy: %0.4f.' % accuracy)

# time used
time_end=time.time()
print('Time to classify: %0.2f minuites.' % ((time_end-time_start)/60))

C:\Users\chocolate\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Accuracy = 90.3333% (271/300) (classification)
Time to classify: 0.01 minuites.


In [1]:
import tensorflow as tf 
tf.reset_default_graph()
sess = tf.InteractiveSession()
tf.device("/gpu:1")
# prepare the data
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST/',one_hot=True)

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 784])
y_ = tf.placeholder("float", [None, 10])
# variables
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x,W) + b)

# first convolutinal layer
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# densely connected layer
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdagradOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = sess.run(accuracy,feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, train accuracy %g" %(i, train_accuracy))
    sess.run(train_step,feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

print("test accuracy %g" % accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))
sess.close()

C:\Users\chocolate\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, train accuracy 0.08
step 100, train accuracy 0.44
step 200, train accuracy 0.66
step 300, train accuracy 0.76
step 400, train accuracy 0.72
step 500, train accuracy 0.78
step 600, train accuracy 0.78
step 700, train accuracy 0.78
step 800, train accuracy 0.76
step 900, train accuracy 0.82
step 1000, train accuracy 0.76
step 1100, train accuracy 0.84
step 1200, train accuracy 0.88
step 1300, train accuracy 0.84
step 1400, train accuracy 0.74
step 1500, train accuracy 0.76
step 1600, train accuracy 0.86
step 1700, train accuracy 0.82
step 1800, train accuracy 0.76
step 1900, train accuracy 0.88
step 2000, train accuracy 0.74
step 2100, train accuracy 0.8
step 2200, train accuracy 0.84
step 2300, train accuracy 0.9
step 2400, train accur

KeyboardInterrupt: 